In [0]:
dbutils.widgets.text("db", "mycatalog", "Database")
catelog = dbutils.widgets.get("db")
dbutils.widgets.text("schema", "hp_prd_data", "Schema")
schema = dbutils.widgets.get("schema")
dbutils.widgets.text("Source_table", "uszips")
dbutils.widgets.text("Target_table", "generated_urls")
dbutils.widgets.text("api_url", "")
dbutils.widgets.text("referal_url", "")
dbutils.widgets.text("zip_load_count", '20', "")


In [0]:
import pprint

In [0]:
config_dict = {
    "db": catelog,
    "schema": schema,
    "api_url": dbutils.widgets.get("api_url"),
    "referal_url": dbutils.widgets.get("referal_url"),
    "zip_load_count": dbutils.widgets.get("zip_load_count"),
    "Target_table": dbutils.widgets.get("Target_table"),
    "Source_table": dbutils.widgets.get("Source_table")
}

In [0]:
pprint.pprint(config_dict)

In [0]:
generated_urls = f"""
CREATE OR REPLACE TABLE {config_dict['db']}.{config_dict['schema']}.{config_dict['Target_table']}
PARTITIONED BY (processed_datehour)
AS
SELECT
  zip,
  state_id,
  state_name,
  format_string(
    '{config_dict["api_url"]}',
    zip
  ) AS api_url,
  format_string(
   '{config_dict["referal_url"]}',
    zip, state_id, state_name
  ) as refereral_url,
  date_format(current_timestamp(), 'yyyy-MM-dd-HH') AS processed_datehour
FROM (
    SELECT 
      zip,
      state_id,
      REPLACE(state_name, ' ', '-') AS state_name, 
      population
    FROM {config_dict['db']}.{config_dict['schema']}.{config_dict['Source_table']}
    WHERE population IS NOT NULL 
    ORDER BY rand()
    LIMIT {config_dict['zip_load_count']}  
) AS sub
"""



In [0]:
spark.sql(generated_urls)